In [ ]:
# Kara Martin
# CS 492 - Computer Security
# Implementation of "Tiny Encryption Algorithm"
# Written 14 February 2021

# Slice up a binary value into equal-length subsections. 
# Not guaranteed that most-significant bit is a 1, so 
# Ask user for size of binary number container.
def yieldEqualSubsections(userInput, numberOfSubsections, size):
    if userInput == 0 or size == 0: 
        return 0
    elif userInput == 1:
        return [userInput]
    #length = binaryLength(userInput) # Cannot do log2(0)
    
    if size % numberOfSubsections != 0:
        print("Error: Pick subsection length that is a factor of container length.")
    else: 
        fullMask = bin(2**size - 1)
        step = int(size / numberOfSubsections) # we're sure this is an integer
        result = []
        counter = 0 
        for i in range(numberOfSubsections):
            startMask = 2**(size - counter) - 1
            stopMask = 2**(size - counter - step) - 1
            finalMask = startMask - stopMask
            result.append((userInput & finalMask) >> size - counter - step)
            counter += step
        return result
        
# In a 32-bit register, higher power terms are lost so 
# we need to simulate this behavior. 
def keepAt32bits(y):
    if y >= 2**32 or abs(y) > 2**32:
        mask = 2**32 - 1 
        a = bin(y)[2:]
        b = bin(y & mask)[2:]
        return y & mask 
    return y

# Some helper methods, mainly for printing 
def bin32(num):
    return f'0b{abs(num):0>32b}'

def hex8(num):
    return f'0x{abs(num):0>8x}'

# yield two's complement of n if negative
def tc(n):
    # negative
    ext = "0" if n & 2**31 or n < 0 else "1"
        
    b = ""
    for a in bin(abs(n))[2:]:
        if a == "1":
            b += "0"
        else:
            b += "1"
    signExtension = ext * (32 - len(b))
    return int(signExtension + b, 2) + 1

# Pretty print in binary, hex, decimal
def pprint(label, num):
    if num & 2**31:
        print(f'{label:>16}:\t {bin32(num)} \t {hex8(num)} \t {-tc(num)}')
    else: 
        print(f'{label:>16}:\t {bin32(num)} \t {hex8(num)} \t {num}')
    
# Forward direction of TEA
def encryptTEA(L, R, K):
    delta = 0x9e3779b9 # 2^32 / phi, key scheduling constant
    sum = 0 
    for i in range(32):
        sum += delta
        L += int(bin32(((R<<4) + K[0]) ^ (R + sum) ^ ((R>>5) + K[1]))[-32:], 2)
        L = keepAt32bits(L)
        R += int(bin32(((L<<4) + K[2]) ^ (L + sum) ^ ((L>>5) + K[3]))[-32:], 2)        
        R = keepAt32bits(R)
    return (L, R)

# Reverse direction of TEA
def decryptTEA(L, R, K):
    delta = 0x9e3779b9 # 2^32 / phi, key scheduling constant
    sum = delta << 5
    for i in range(32):
        R += tc(int(bin32( ((L<<4) + K[2]) ^ (L + sum) ^ ((L>>5) + K[3]))[-32:], 2))
        R = keepAt32bits(R)
        L += tc(int(bin32(((R<<4) + K[0]) ^ (R + sum) ^ ((R>>5) + K[1]))[-32:], 2))
        L = keepAt32bits(L)
        sum -= delta
    return (L, R)

# Part 3: Explain how to encrypt/decrypt multiple blocks of plaintext with CBC (Cipher Block Chaining)
# First, we need a random initialization vector (IV). 
# Our basic structure for CBC is:
# ENCRYPTION:                       DECRYPTION:
# C_0 = E(IV ^ P_0, K)             P_0 = IV ^ D(C_0, K)
# C_1 = E(C_0 ^ P_1, K)            P_1 = C_0 ^ D(C_1, K)
#         .                                  .
#         .                                  .
#         .                                  .
# So to write this for my implementation of TEA, I would first compute C_0 with the random IV and P_0 = (L_0, R_0).
# Where P_0 is split into left and right halves. With C_0 computed, we can compute C_1. 
# In this way we can completely encrypt multiple blocks of plaintext with TEA.
# To decrypt, we start with decrypting C_0 in the same way as a single block of plaintext, then XOR with the IV. 
# For subsequent cipher blocks, we decrypt as normal and then XOR with the ciphertext of the previous plaintext. 
# In this way we can completely decrypt the entire message. 

In [ ]:
def fullTEA(key, plaintext):
    K = yieldEqualSubsections(key, 4, 128)
    for i, x in enumerate(K):
        label = f'K[{i}]:'
        if x & 2**31:
            print(f' {label:>16}\t {bin32(x)} \t {hex8(x)} \t -{tc(x)}')
        else: 
            print(f' {label:>16}\t {bin32(x)} \t {hex8(x)} \t {x}')

    print("============================================")
    PSplit = yieldEqualSubsections(plaintext, 2, 64)
    L = PSplit[0]
    pprint('L', L)
    R = PSplit[1]
    pprint('R', R)
    print(f'{"Full Plaintext:":>17}\t {hex8((L << 32) + R)}')
    print("============================================")

    cipherL, cipherR = encryptTEA(L, R, K)
    pprint('cipherL', cipherL)
    pprint('cipherR', cipherR)
    print(f'{"Full Cipher:":>17}\t {hex8((cipherL << 32) + cipherR)}')
    print()

    decryptL, decryptR = decryptTEA(cipherL, cipherR, K)
    pprint('decryptL', decryptL)
    pprint('decryptR', decryptR)
    print(f'{"Full Decrypted:":>17}\t {hex8((decryptL << 32) + decryptR)}')
    print(f'{"Full Plaintext:":>17}\t {hex8((L << 32) + R)}')
    print("Does decrypted text match original plaintext? ...", decryptL == L and decryptR == R)
    print("============================================")


In [ ]:
## Test 1
fullTEA(0xa56babcdf000ffffffffffffabcdef01, 0x123456789abcdef)

In [ ]:
## Test 2
fullTEA(0xa56babcdffffffffffffffffabcdef01, 0x123456789abcdef)

In [ ]:
## Test 3
fullTEA(0xa56babcdffabffffffffffffabcdef01, 0x123456789abcdef)

In [ ]:
## Homework Problem 
fullTEA(0xAF6BABCDEF00F000FEAFAFAFACCDEF01, 0x01CA45670CABCDEF)